In [ ]:
# https://www.youtube.com/watch?v=YSLVAxgclCo
#https://github.com/feitgemel/Open-CV/blob/main/Detect%20moving%20objects%20in%20%20fixed%20background/Detect-moving-objects-in-a-video-with-fixed-background.py




Segundo Cunha (2013), o modelo de mistura de gaussianas (MoG – Mixture of
Gaussian) é o modelo mais usual em processamento de imagens para modelagem do fundo
em imagens de vídeo. O método básico da mistura de gaussianas busca modelar os pixels
do fundo através de uma função de distribuição de probabilidades aproximada por uma
soma de K distribuições gaussianas ponderadas (KAEWTRAPULPONG; BOWDEN,
2001). Os pesos de cada gaussiana representam a proporção de tempo em que cada cor
esteve presente na cena (RUAS; BENSO, 2006). Como as prováveis cores do background
tendem a ficar estáticas e permanecer por mais tempo no vídeo, os pixels de background
são identificados ao verificar-se quais misturas de distribuições possuem peso maiores
(MARCOMINI, 2018).
Neste trabalho, o algoritmo utilizado para fazer a segmentação da imagem é o
MoG2 (Mixture of Gaussian 2 ), que é uma variação do algoritmo MoG. Ele foi criado
baseando-se em dois artigos de Zivkovic (2004 e 2006). A diferença básica entre o MoG e o
MoG2 está no uso das distribuições gaussianas. Enquanto algoritmo MoG usa apenas um
valor K para todos os pixels do background, algoritmo MoG2 usa um número adequado de
distribuições gaussianas para cada pixel, proporcionando uma melhor adaptação à variação
da cena devido às mudanças de iluminação, e também há a opção de detectar de sombras.
Para o algoritmo de subtração de fundos (MoG2) ser iniciado, são pedidos três
parâmetros:
• history: define a quantidade de quadros anteriores que serão considerados para gerar
o modelo de background;
• varThreshold: retorna o limite de variância para o modelo de pixel correspondido.
28 Capítulo 1. Referenciais Teóricos
É o principal limiar no quadrado da distância de Mahalanobis para decidir se a
amostra é bem descrita pelo modelo de background ou não;
• detectShadows: retorna a flag de detecção de sombras, caso seja verdadeiro o algoritmo
detecta as sombras e as marca.
Caso esses parâmetros não sejam passados, ele será iniciado com valores padrão.
Dependendo das condições do vídeo pode-se fazer um ajuste fino desses parâmetros
para que a segmentação da imagem seja feita da melhor forma. O parâmetro history com
valores baixos faz com que o algoritmo comporte-se melhor caso haja mudanças bruscas
na luminosidade, tendo uma adaptação mais rápida, porém pode fazer com que ele fique
muito sensível a mudanças na cena, podendo fazer com que objetos verdadeiros que se
movem lentamente façam parte do fundo; valores altos fazem com que o modelo de fundo
fique mais estável, mas requer mais tempo para isso, fazendo com que mudanças no fundo
sejam calculadas de maneira mais suave, porém demorando mais para se estabilizar em
caso de mudanças bruscas de luminosidade, podendo fazer com que o background se torne
um objeto. O parâmetro varThreshold com valores baixos pode fazer com que parte do
background se torne objeto, ou seja, apareçam falsos objetos na cena; já este parâmetro
com valores mais altos pode fazer com que objetos verdadeiros façam parte do background.
Caso o parâmetro detectShadows seja acionado, haverá um impacto no desempenho do
algoritmo, pois vai exigir um maior processamento.

In [ ]:
!pip install opencv-python


In [1]:
# Para o presente trabalho, utilizarei o modelo MOG2
#https://www.anpet.org.br/anais32/documentos/2018/Trafego%20Urbano%20e%20Rodoviario/Trafego%20em%20Vias%20Urbanas%20II/3_569_AC.pdf
#Basicamente ele é utilizado para a verificação de trafego urbano, onde a câmera encontra-se parada enquanto os carros
# se movimentam. Assim ele dete 
# Importa as bibliotecas necessárias: OpenCV (cv2) para processamento de vídeo e NumPy para manipulação de arrays.

import cv2
import numpy as np

cap = cv2.VideoCapture("cars.mp4")

# Cria um objeto de subtração de plano de fundo utilizando o método MOG2 (Mixture of Gaussians), com um histórico de 2 frames.
backgroundObject = cv2.createBackgroundSubtractorMOG2(history=2)

#Cria um kernel 3x3 composto por uns (pixels brancos) que será usado para operações morfológicas de erosão e dilatação.
kernel = np.ones((3,3),np.uint8)

#Inicializa uma segunda variável de kernel como None. Isso será usado posteriormente.
kernel2 = None

#Inicia um loop infinito para processar o vídeo frame a frame.
#Lê um frame do vídeo. A variável ret indica se a leitura do frame foi bem-sucedida, e frame contém o frame lido.
#Se a leitura do frame falhar (quando ret é falso), o loop é interrompido.
while True:
    ret , frame = cap.read()
    if not ret :
        break
#Aplica o algoritmo de subtração de plano de fundo ao frame atual para obter uma máscara de plano de fundo.
#Aplica um limiar na máscara para binarizá-la, convertendo-a em uma imagem preto e branco.
#Aplica uma operação de erosão na máscara para remover ruídos menores.
#Aplica uma operação de dilatação na máscara para preencher buracos e suavizar as bordas.


    fgmask = backgroundObject.apply(frame)
    _, fgmask = cv2.threshold(fgmask ,20 , 255 , cv2.THRESH_BINARY)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    fgmask = cv2.dilate(fgmask,kernel2 , iterations=6  )


    #Encontra os contornos na máscara de plano de fundo para detectar objetos em movimento.

    countors, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #Faz uma cópia do frame original para desenhar retângulos e textos sobre ele.
    frameCopy = frame.copy()

    # Inicia um loop para percorrer todos os contornos encontrados.
    #Verifica se a área do contorno é maior que 20000 pixels. Isso é feito para filtrar objetos pequenos, considerando que carros são geralmente maiores.

    for cnt in countors:
        if cv2.contourArea(cnt) > 20000:

            # Obtém as coordenadas do retângulo delimitador ao redor do contorno.
            x , y, width , height = cv2.boundingRect(cnt)

            # Desenha um retângulo vermelho em volta do objeto detectado.
            cv2.rectangle(frameCopy, (x,y), (x+width, y+ height) , (0,0,255), 2)

            # Escreve um texto indicando a detecção do carro próximo ao retângulo.
            cv2.putText(frameCopy ,"Car detected", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3 , (0,255,0), 1, cv2.LINE_AA)

#Aplica a máscara de plano de fundo ao frame original para destacar apenas os objetos detectados.

    forground = cv2.bitwise_and(frame,frame , mask = fgmask)

    # Empilha horizontalmente o frame original, o frame com apenas os objetos detectados e o frame com retângulos e textos sobre os objetos.
    stacked = np.hstack((frame,forground,frameCopy))

    #Exibe a imagem empilhada com uma redução de escala de 40% na janela chamada "stacked".
    cv2.imshow("stacked", cv2.resize(stacked,None,fx=0.4, fy=0.4))

    #cv2.imshow("forground", forground)
    #cv2.imshow("frameCopy", frameCopy)
    #cv2.imshow("fgmask", fgmask)
    #cv2.imshow("img", frame)

    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

In [2]:
# Importa as bibliotecas necessárias: OpenCV (cv2) para processamento de vídeo e NumPy para manipulação de arrays.

import cv2
import numpy as np

cap = cv2.VideoCapture("india.mp4")

# Cria um objeto de subtração de plano de fundo utilizando o método MOG2 (Mixture of Gaussians), com um histórico de 2 frames.
backgroundObject = cv2.createBackgroundSubtractorMOG2(history=2)

#Cria um kernel 3x3 composto por uns (pixels brancos) que será usado para operações morfológicas de erosão e dilatação.
kernel = np.ones((3,3),np.uint8)

#Inicializa uma segunda variável de kernel como None. Isso será usado posteriormente.
kernel2 = None

#Inicia um loop infinito para processar o vídeo frame a frame.
#Lê um frame do vídeo. A variável ret indica se a leitura do frame foi bem-sucedida, e frame contém o frame lido.
#Se a leitura do frame falhar (quando ret é falso), o loop é interrompido.
while True:
    ret , frame = cap.read()
    if not ret :
        break
#Aplica o algoritmo de subtração de plano de fundo ao frame atual para obter uma máscara de plano de fundo.
#Aplica um limiar na máscara para binarizá-la, convertendo-a em uma imagem preto e branco.
#Aplica uma operação de erosão na máscara para remover ruídos menores.
#Aplica uma operação de dilatação na máscara para preencher buracos e suavizar as bordas.


    fgmask = backgroundObject.apply(frame)
    _, fgmask = cv2.threshold(fgmask ,20 , 255 , cv2.THRESH_BINARY)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    fgmask = cv2.dilate(fgmask,kernel2 , iterations=6  )


    #Encontra os contornos na máscara de plano de fundo para detectar objetos em movimento.

    countors, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #Faz uma cópia do frame original para desenhar retângulos e textos sobre ele.
    frameCopy = frame.copy()

    # Inicia um loop para percorrer todos os contornos encontrados.
    #Verifica se a área do contorno é maior que 20000 pixels. Isso é feito para filtrar objetos pequenos, considerando que carros são geralmente maiores.

    for cnt in countors:
        if cv2.contourArea(cnt) > 20000:

            # Obtém as coordenadas do retângulo delimitador ao redor do contorno.
            x , y, width , height = cv2.boundingRect(cnt)

            # Desenha um retângulo vermelho em volta do objeto detectado.
            cv2.rectangle(frameCopy, (x,y), (x+width, y+ height) , (0,0,255), 2)

            # Escreve um texto indicando a detecção do carro próximo ao retângulo.
            cv2.putText(frameCopy ,"Car detected", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3 , (0,255,0), 1, cv2.LINE_AA)

#Aplica a máscara de plano de fundo ao frame original para destacar apenas os objetos detectados.

    forground = cv2.bitwise_and(frame,frame , mask = fgmask)

    # Empilha horizontalmente o frame original, o frame com apenas os objetos detectados e o frame com retângulos e textos sobre os objetos.
    stacked = np.hstack((frame,forground,frameCopy))

    #Exibe a imagem empilhada com uma redução de escala de 40% na janela chamada "stacked".
    cv2.imshow("stacked", cv2.resize(stacked,None,fx=0.4, fy=0.4))

    #cv2.imshow("forground", forground)
    #cv2.imshow("frameCopy", frameCopy)
    #cv2.imshow("fgmask", fgmask)
    #cv2.imshow("img", frame)

    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

cap = cv2.VideoCapture("cars.mp4")

backgroundObject = cv2.createBackgroundSubtractorMOG2(history=2)

kernel = np.ones((3,3),np.uint8)

kernel2 = None

while True:
    ret , frame = cap.read()
    if not ret :
        break


    fgmask = backgroundObject.apply(frame)
    _, fgmask = cv2.threshold(fgmask ,20 , 255 , cv2.THRESH_BINARY)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    fgmask = cv2.dilate(fgmask,kernel2 , iterations=6  )



    countors, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    frameCopy = frame.copy()


    for cnt in countors:
        if cv2.contourArea(cnt) > 20000:

            x , y, width , height = cv2.boundingRect(cnt)

            cv2.rectangle(frameCopy, (x,y), (x+width, y+ height) , (0,0,255), 2)

            cv2.putText(frameCopy ,"Car detected", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3 , (0,255,0), 1, cv2.LINE_AA)


    forground = cv2.bitwise_and(frame,frame , mask = fgmask)

    stacked = np.hstack((frame,forground,frameCopy))

    cv2.imshow("stacked", cv2.resize(stacked,None,fx=0.4, fy=0.4))


    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()